In [4]:
import requests, time, datetime
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

def my_random_choice1(l,s=0,e=5):
    n = np.random.choice(range(s,e),1)[0]
    return ','.join(np.random.choice(l,n))
def my_random_choice2(l,s=0,e=5):
    n = np.random.choice(range(s,e),1,[0.4,0.3,0.1,0.1,0.1])[0]
    return ','.join(np.random.choice(l,n))
def extract_card_name(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('li', class_='item')
    names = [i.find('b', class_='name').text for i in items]
    image_info = [i.find('img',class_='img')['src'] for i in items]
    
    return [(n,i) for n,i in zip(names,image_info)]

company_Code = ['SH','HD','SS','KB','LO','SK','WR','CT','NH','IB']
brand_Names = ['mastercard','visa','unionpay','jcb','bc','amex']
benefit_Category_Ids = [1,12,7,16,14,13,17,15,8,10,9,11,18,19,4,5,6,2,3,21,20,22,52,62,63,83,103,123,143,163,1001,1002,1003]
benefit_Category_Ids = [str(i) for i in benefit_Category_Ids]
annualFee = ['','&maxAnnualFee=10000','&maxAnnualFee=30000','&maxAnnualFee=100000','&minAnnualFee=100000']
sortMethod = ['&sortMethod=qc','&sortMethod=ri']

In [5]:
start = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
total_num = 364
name_list = []
n = len(name_list)
root_url = 'https://card-search.naver.com/list?'
end_url = '&query=%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%EC%B6%94%EC%B2%9C&isRefetch=true'
i = 0
while (total_num - n) > 0:
    
    CC = my_random_choice2(company_Code) #company_Code
    BN = my_random_choice1(brand_Names) #brand_Names
    BCI = my_random_choice1(benefit_Category_Ids) #benefit_Category_Ids
    AF = np.random.choice(annualFee,1)[0] # annual_fee
    SM = np.random.choice(sortMethod,1)[0] # sort method
    content_url = f'companyCode={CC}&brandNames={BN}&benefitCategoryIds={BCI}{AF}{SM}'
    temp_url = root_url + content_url + end_url
    temp_name = extract_card_name(temp_url)
    
    name_list.extend(temp_name)
    name_list = list(set(name_list))
    n = len(name_list)
    
    i += 1
    time.sleep(0.1)
#     if len(temp_num)>=3:
#         print(temp_url)
    print(f'현재 가져온 카드 갯수 : {n}/{total_num}, 시도한 크롤링 횟수 : {i},   시작시간 : {start}', end='\r')

# 1시간정도 돌리면 360개정도는 모아집니다.
# case나눠주는거를 10개 미만으로 나누어지게 disjoint하게 다 돌릴 수 있으면 좋을 것 같은데 방법을 아직 찾지 못했습니다.

KeyboardInterrupt: 

In [6]:
card_name = [a[0] for a in name_list]
card_tag = [a[1].split('/')[5] for a in name_list]

AMF_list, CH_list, BE_list = [], [], []

start_url = 'https://card-search.naver.com/item?'
end_url = '&pquery=%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%EC%B6%94%EC%B2%9C'
for i in range(len(card_name)):
    content_url = f'cardAdId={card_tag[i]}&query={card_name[i]}'
    temp_url = start_url + content_url + end_url
    response = requests.get(temp_url)

    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
#     print(f'카드이름 : {card_name[i]}, url : {temp_url}')

    # AMF: annual membership fee, CH: credit history, BE: benefit
    box = soup.find('dl', class_='list')
    AMF = box.find('dd',class_='desc as_annualFee').text
    CH = ''
    BE = ''
    
    AMF_list.append(AMF)
    
    time.sleep(0.1)
    print(f'카드 정보 긁어오는중...  {i}/{len(card_name)-1}', end='\r')

In [7]:

df = pd.DataFrame({'카드이름':card_name,
              '연회비':AMF_list,
              '기준실적':'',
              '혜택요약':''})
df

,카드이름,연회비,기준실적,혜택요약
0,LOCA CLASSIC (로카 클래식),"국내 2만원,해외 2만원",,
1,롯데백화점 FLEX카드,"국내 10만원,해외 10만원",,
2,참! 좋은 친구 청년동행카드,"국내 2천원,해외 5천원",,
3,위메프우리카드,"국내 1만원,해외 1만2천원신규 최대 1만2천원 지급",,
4,신한 Hi-Point 카드,해외 8천원신규 최대 1만5천원 지급,,
...,...,...,...,...
263,카드의정석 댕댕냥이,"국내 1만원,해외 1만2천원신규 최대 1만2천원 지급",,
264,1Q Special+ (원큐 스페셜플러스),해외 1만원,,
265,KB국민 굿데이올림카드,"국내 1만5천원,해외 2만원신규 최대 1만5천원 지급",,
266,Smart Any (스마트애니),"국내 5천원,해외 5천원신규 최대 5천원 지급",,
